## Precarga de librerias y funciones

In [1]:
%run "../recurrentes.ipynb"

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [2]:
%run "../funciones.ipynb"

#### Excel

In [3]:
# Activar Excel de seguimiento de medidas? 

excel = True
if excel == True:
    # Ingresar path excel para anotar resultados
    path_excel = base_path_l + r'\resultados_backtesting.xlsx'
    date = datetime.now().strftime("%d-%m-%Y %H-%M-%S")
    

In [4]:
# 2 - Cargar datos
# Opciones:
# a) dataset_clase_ternaria_l
# b) dataset_lags_clase_ternaria_l
# c) dataset_lags_deltas_y_clase_ternaria_l


dataset = dataset_clase_ternaria_l
dataset_name = os.path.basename(dataset)
df_train = pd.read_parquet(dataset)


## Formateo pre modelo

In [5]:
if 'T_Visa_normal' in df_train.columns:
    df_train['T_Visa_normal'] = df_train['T_Visa_normal'].astype(bool)
if 'T_Master_normal'in df_train.columns:
    df_train['T_Master_normal'] = df_train['T_Master_normal'].astype(bool)


In [6]:
# 3 - parametros para modelo
ganancia_acierto = 273000
costo_estimulo = 7000

mes_train = 202104
mes_test = 202106


In [7]:
data = df_train
data.shape

(4735593, 155)

In [8]:

data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [9]:
data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)

In [10]:
df_train['foto_mes'].unique()

array([201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004,
       202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,
       202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108],
      dtype=int64)

In [11]:
# Especificar mes de train y test

df_train = data[data['foto_mes']<=mes_train]
df_test = data[data['foto_mes']==mes_test]


In [12]:
print(data.shape)  # Número de filas y columnas
print(data.info(memory_usage='deep'))

(4735593, 157)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4735593 entries, 0 to 4735592
Columns: 157 entries, numero_de_cliente to clase_binaria
dtypes: float64(92), int32(1), int64(63), object(1)
memory usage: 5.8 GB
None


In [13]:
clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']

In [14]:
w_train = df_train.loc[X_train.index, 'clase_peso']


In [15]:
X_train

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,249221109,201901,1,0,1,59,276,7597.55,47433.58,5654.59,...,0.0,1.0,0.0,-44919.57,3.23,7136.0,24336.99,13.0,0.0,1466.25
1,249221468,201901,1,0,1,51,13,738.16,-325.09,491.31,...,0.0,1.0,0.0,-30184.75,15.24,393.0,16345.79,8.0,0.0,1548.36
2,249223005,201901,1,0,1,46,178,1014.31,9434.15,417.36,...,0.0,1.0,0.0,-3730.14,0.00,2755.0,1181.65,3.0,0.0,3917.82
3,249228180,201901,1,0,1,64,296,1028.33,6873.80,1129.43,...,0.0,1.0,0.0,-13196.25,0.00,8194.0,1892.04,1.0,0.0,2017.56
4,249232117,201901,1,0,1,77,349,11617.61,43291.55,10726.08,...,0.0,1.0,0.0,-22073.65,0.00,7876.0,21902.85,6.0,0.0,891.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4075495,1592771867,202104,1,0,0,28,1,35.79,35.79,0.00,...,NaN,2.0,0.0,NaN,NaN,22.0,NaN,NaN,NaN,0.00
4075496,1592792217,202104,1,0,0,18,1,4.85,4.85,0.00,...,NaN,2.0,0.0,NaN,NaN,18.0,NaN,NaN,NaN,0.00
4075497,1592797828,202104,0,0,0,36,1,47.62,47.62,0.00,...,NaN,2.0,0.0,NaN,NaN,17.0,NaN,NaN,NaN,0.00
4075498,1592799066,202104,0,0,0,37,1,0.00,0.00,0.00,...,NaN,2.0,0.0,NaN,NaN,19.0,NaN,NaN,NaN,0.00


In [16]:
X_train.fillna(0, inplace=True)
Y_train.fillna(0, inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_13800\1651022412.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train.fillna(0, inplace=True)


In [17]:
Y_train.unique()

array([0, 1])

In [ ]:
# Separar train y test según 'foto_mes'
from imblearn.under_sampling import ClusterCentroids


# Aplicar Tomek Links respetando las series temporales
# Crear una ventana acumulativa para mantener la dependencia temporal
# Inicializar las listas para almacenar los datos remuestreados
X_resampled_list = []
y_resampled_list = []

for i in range(201901, 202106):
    mask = df_train['foto_mes'] == i
    X_window = X_train[mask].copy()
    y_window = Y_train[mask].copy()
    
    # Verificar si y_window tiene más de una clase
    if y_window.nunique() > 1:
        # Aplicar Tomek Links
        cc = ClusterCentroids(random_state=0)
        X_resampled, y_resampled = cc.fit_resample(X_train, Y_train)
    else:
        # Si solo hay una clase, conservar los datos originales
        X_resampled_window = X_window
        y_resampled_window = y_window
    
    # Almacenar los datos resampleados
    X_resampled_list.append(X_resampled_window)
    y_resampled_list.append(y_resampled_window)

# Concatenar los datos resampleados
X_resampled = pd.concat(X_resampled_list)
y_resampled = pd.concat(y_resampled_list)



c:\Users\Admin\.conda\envs\datascience\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [20]:
X_train = X_resampled 
Y_train = y_resampled

In [31]:
w_train = df_train.loc[X_train.index, 'clase_peso']

In [32]:
w_train

0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
163095    1.0
163096    1.0
163097    1.0
163098    1.0
163099    1.0
Name: clase_peso, Length: 4062281, dtype: float64

In [33]:
w_test = df_test.loc[Y_test.index, 'clase_peso']
test_data = lgb.Dataset(X_test, label=Y_test, weight=w_test)
test_data

## Cross-Validation

In [16]:
# Ordenar entradas en df_train en base a la columna foto_mes de meses más antiguos a más recientes
X_train = X_train.sort_values('foto_mes', ascending=True)

In [20]:
semillas

[400009, 500009, 500011, 500021, 600009]

In [22]:
tscv = TimeSeriesSplit(n_splits=5)
folds = tscv.split(X_train)

lista_cv = []

params = {
    'objective': 'binary',  # Puedes cambiar esto si tu problema es multiclase u otro tipo
    'metric': 'binary_logloss',  # Cambia el metric si es necesario
    }

train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)




# Pasar las divisiones a lgb.cv
cv_results = lgb.cv(
    params,
    train_data,
    num_boost_round=300,
    feval=lgb_gan_eval,
    folds=folds,  # Aquí especificamos las divisiones temporales
    seed=s,
)

cv_results

max_gan = max(cv_results['valid gan_eval-mean'])
best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

final =  max_gan * 5
lista_cv.append(final)



[LightGBM] [Info] Number of positive: 7210, number of negative: 672040
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.071578 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 679250, number of used features: 154
[LightGBM] [Info] Number of positive: 76290, number of negative: 1282210
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.134004 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 1358500, number of used features: 154
[LightGBM] [Info] Number of positive: 294796, number of negative: 1742954
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhea

In [27]:
tscv = TimeSeriesSplit(n_splits=5)
folds = tscv.split(X_train)


params = {
    'objective': 'binary',  # Puedes cambiar esto si tu problema es multiclase u otro tipo
    'metric': 'binary_logloss',  # Cambia el metric si es necesario
    }

train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)




# Pasar las divisiones a lgb.cv
cv_results = lgb.cv(
    params,
    train_data,
    num_boost_round=300,
    feval=lgb_gan_eval,
    folds=folds,  # Aquí especificamos las divisiones temporales
    seed=semillas[1],
)

cv_results

max_gan = max(cv_results['valid gan_eval-mean'])
best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

final_1 =  max_gan * 5
lista_cv.append(final_1)

[LightGBM] [Info] Number of positive: 7210, number of negative: 672040
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.089561 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 679250, number of used features: 154
[LightGBM] [Info] Number of positive: 76290, number of negative: 1282210
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.130294 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 1358500, number of used features: 154
[LightGBM] [Info] Number of positive: 294796, number of negative: 1742954
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhea

In [23]:
tscv = TimeSeriesSplit(n_splits=5)
folds = tscv.split(X_train)


params = {
    'objective': 'binary',  # Puedes cambiar esto si tu problema es multiclase u otro tipo
    'metric': 'binary_logloss',  # Cambia el metric si es necesario
    }

train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)




# Pasar las divisiones a lgb.cv
cv_results = lgb.cv(
    params,
    train_data,
    num_boost_round=300,
    feval=lgb_gan_eval,
    folds=folds,  # Aquí especificamos las divisiones temporales
    seed=semillas[2],
)

cv_results

max_gan = max(cv_results['valid gan_eval-mean'])
best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

final_2 =  max_gan * 5
lista_cv.append(final_2)

[LightGBM] [Info] Number of positive: 7210, number of negative: 672040
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.062586 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 679250, number of used features: 154
[LightGBM] [Info] Number of positive: 76290, number of negative: 1282210
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.129290 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 1358500, number of used features: 154
[LightGBM] [Info] Number of positive: 294796, number of negative: 1742954
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhea

In [24]:
tscv = TimeSeriesSplit(n_splits=5)
folds = tscv.split(X_train)




params = {
    'objective': 'binary',  # Puedes cambiar esto si tu problema es multiclase u otro tipo
    'metric': 'binary_logloss',  # Cambia el metric si es necesario
    }

train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)




# Pasar las divisiones a lgb.cv
cv_results = lgb.cv(
    params,
    train_data,
    num_boost_round=300,
    feval=lgb_gan_eval,
    folds=folds,  # Aquí especificamos las divisiones temporales
    seed=semillas[3],
)

cv_results

max_gan = max(cv_results['valid gan_eval-mean'])
best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

final_3 =  max_gan * 5
lista_cv.append(final_3)

[LightGBM] [Info] Number of positive: 7210, number of negative: 672040
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.074897 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 679250, number of used features: 154
[LightGBM] [Info] Number of positive: 76290, number of negative: 1282210
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.156746 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 1358500, number of used features: 154
[LightGBM] [Info] Number of positive: 294796, number of negative: 1742954
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhea

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
folds = tscv.split(X_train)



params = {
    'objective': 'binary',  # Puedes cambiar esto si tu problema es multiclase u otro tipo
    'metric': 'binary_logloss',  # Cambia el metric si es necesario
    }

train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)




# Pasar las divisiones a lgb.cv
cv_results = lgb.cv(
    params,
    train_data,
    num_boost_round=300,
    feval=lgb_gan_eval,
    folds=folds,  # Aquí especificamos las divisiones temporales
    seed=semillas[4],
)

cv_results

max_gan = max(cv_results['valid gan_eval-mean'])
best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

final_4 =  max_gan * 5
lista_cv.append(final_4)

[LightGBM] [Info] Number of positive: 7210, number of negative: 672040
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.074043 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 679250, number of used features: 154
[LightGBM] [Info] Number of positive: 76290, number of negative: 1282210
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.440371 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 1358500, number of used features: 154
[LightGBM] [Info] Number of positive: 294796, number of negative: 1742954
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.191630 seconds.
You can set `force_row_wise=t

In [28]:
finalisimo =  sum(lista_cv)/ len(lista_cv)  
finalisimo

10737406704.0

In [ ]:
# RESULTADOS:
# 1. Modelo Base = 10737406704.0

## Predicción y scoring

In [ ]:
dicc_medidas = {}


In [1]:
def calcular_medidas(X_train, Y_train, w_train, X_test, Y_test, w_test):

    lista_medidas = [accuracy_score, precision_score, recall_score, f1_score, roc_auc_score]
    train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)
    train_data = lgb.Dataset(X_test, label=X_test, weight=w_test)
    df_voting = pd.DataFrame()

    for semilla in semillas:
        params = {
            'objective': 'binary',  
            'metric': 'binary_logloss',  
            'seed': semilla, }
        model_default = lgb.train(params, train_data)   
        
        df_voting[f'semilla_{semilla}'] = model_default.predict(X_test)

    df_voting['promedio'] = df_voting.mean(axis=1)
    y_pred_default = df_voting['promedio']
    y_pred_labels = (y_pred_default >= 0.025).astype(int)
    dicc_medidas = {}
    
    for medida in lista_medidas:
     dicc_medidas[medida.__name__] = medida(Y_test, y_pred_labels)
    ganancia = lgb_gan_eval(y_pred_default, test_data)
    dicc_medidas['ganancia'] = ganancia[1]
    
    return dicc_medidas
    

#### Medidas

In [38]:
y_pred_default = df_voting['promedio']

In [ ]:
if excel == True:
    dicc_medidas['fecha'] = date
    dicc_medidas['dataset'] = dataset_name
    

In [41]:
for medida in lista_medidas:
     dicc_medidas[medida.__name__] = medida(Y_test, y_pred_labels)
print(dicc_medidas)

{'fecha': '20-11-2024 20-35-46', 'dataset': 'competencia_02_ct.parquet', 'accuracy_score': 0.9173560736553531, 'precision_score': 0.08968483395614468, 'recall_score': 0.6401610468042275, 'f1_score': 0.15732838589981446, 'roc_auc_score': 0.7804492407495098}


In [42]:
ganancia = lgb_gan_eval(y_pred_default, test_data)
dicc_medidas['ganancia'] = ganancia[1]

In [45]:
cantidad_columnas = df_train.shape[1]
mes_train = df_train['foto_mes'].max()
mes_test = df_test['foto_mes'].max()
# Agregar nota
consideraciones = 'undersampling con Tomek Links'

dicc_medidas['cantidad_columnas'] = cantidad_columnas
dicc_medidas['mes_train'] = mes_train
dicc_medidas['mes_test'] = mes_test
dicc_medidas['consideraciones'] = consideraciones

In [46]:
dicc_medidas 

{'fecha': '20-11-2024 20-35-46',
 'dataset': 'competencia_02_ct.parquet',
 'accuracy_score': 0.9173560736553531,
 'precision_score': 0.08968483395614468,
 'recall_score': 0.6401610468042275,
 'f1_score': 0.15732838589981446,
 'roc_auc_score': 0.7804492407495098,
 'ganancia': 75285000,
 'cantidad_columnas': 157,
 'mes_train': 202104,
 'mes_test': 202106,
 'consideraciones': 'undersampling con Tomek Links'}

In [47]:
if excel == True:
    
    wb = load_workbook(path_excel)
    ws = wb.active

    header_row = 1  
    column_mapping = {cell.value: cell.column for cell in ws[header_row]}

    # Verificar el mapeo de columnas
    print("Mapeo de columnas:", column_mapping)
    
row_to_update = None

for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=1):
    cell = row[0]  # La primera celda de la fila
    if cell.value is None:  # Si el valor de la celda está vacía
        print(f"Primera celda vacía en la fila {cell.row}")
        row_to_update = cell.row
        break

# Si no se encontró ninguna celda vacía, agregar al final
if row_to_update is None:
    print("No se encontro celda vacía, agregando al final")
    row_to_update = ws.max_row + 1  # Agregar en una nueva fila al final

print(f"Fila a actualizar: {row_to_update}")
fila_actualizar = row_to_update  # Fila donde escribir los datos

for col_name, value in dicc_medidas.items():
    if col_name in column_mapping:  # Asegurarse de que la columna existe
        col_idx = column_mapping[col_name]
        ws.cell(row=fila_actualizar, column=col_idx, value=value)

# Guardar los cambios
wb.save(path_excel)



Mapeo de columnas: {'fecha': 1, 'dataset': 2, 'mes_train': 3, 'mes_test': 4, 'consideraciones': 5, 'ganancia': 6, 'accuracy_score': 7, 'precision_score': 8, 'recall_score': 9, 'f1_score': 10, None: 13, 'cantidad_columnas': 12}
Primera celda vacía en la fila 20
Fila a actualizar: 20
